### Parsing Mujoco env

In [7]:
import copy
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../../../")
from utils.mujoco_parser import MuJoCoParserClass
from utils.util import sample_xyzs,rpy2r,r2quat
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


In [20]:
xml_path = '../../../asset/scene_realworld_wo_shelf.xml'
env = MuJoCoParserClass(name='Place task scene: Office table',rel_xml_path=xml_path,VERBOSE=False, MODE='window')
print(env.MODE)

# Move tables and robot base
env.model.body('base_table').pos = np.array([0,0,0])
env.model.body('avoiding_object_table').pos = np.array([0.38+0.45,0,0])
env.model.body('base').pos = np.array([-0.18,0,0.79])
env.model.body('right_object_table').pos = np.array([-0.05,-0.80,0])
env.model.body('left_object_table').pos = np.array([-0.05,0.80,0])

# Place objects
tray_pos = np.array([ 0.9, 0.0, 0.8])
bowl_red_pos = np.array([ 0.9, 0.3, 0.8])
bowl_blue_pos = np.array([ 0.9, -0.3, 0.8])
env.model.joint(env.model.body('tray_gray').jntadr[0]).qpos0[:3] = tray_pos
env.model.joint(env.model.body('kitchen-bowl-red').jntadr[0]).qpos0[:3] = bowl_red_pos
env.model.joint(env.model.body('kitchen-bowl-blue').jntadr[0]).qpos0[:3] = bowl_blue_pos

env.model.joint(env.model.body('ycb-apple-2').jntadr[0]).qpos0[:3] = bowl_red_pos + np.array([0.0,0.0,0.02])
env.model.joint(env.model.body('ycb-banana-2').jntadr[0]).qpos0[:3] = tray_pos + np.array([0.0,0.03,0.2])
env.model.joint(env.model.body('ycb-banana-2').jntadr[0]).qpos0[3:] = r2quat(rpy2r(np.radians([0, 0, 90])))
env.model.joint(env.model.body('ycb-lemon-2').jntadr[0]).qpos0[:3] = tray_pos + np.array([-0.03,-0.07,0.02])
env.model.joint(env.model.body('ycb-orange-2').jntadr[0]).qpos0[:3] = bowl_blue_pos + np.array([0.0,0.0,0.02])

# Target objects
env.model.joint(env.model.body('ycb-apple').jntadr[0]).qpos0[:3] = np.array([0.1, -0.6, 0.8])
env.model.joint(env.model.body('ycb-banana').jntadr[0]).qpos0[:3] = np.array([-5.0,-1.5,0.2])
env.model.joint(env.model.body('ycb-lemon').jntadr[0]).qpos0[:3] = np.array([0.0, -0.6, 0.8])
env.model.joint(env.model.body('ycb-orange').jntadr[0]).qpos0[:3] = np.array([-0.1, -0.6, 0.8])

joint_names = env.rev_joint_names[:6]
idxs_forward = [env.model.joint(joint_name).qposadr[0] for joint_name in env.joint_names[:6]]
idxs_jacobian = [env.model.joint(joint_name).dofadr[0] for joint_name in env.joint_names[:6]]
list1, list2 = env.ctrl_joint_idxs, idxs_forward
idxs_step = []
for i in range(len(list2)):
    if list2[i] in list1:
        idxs_step.append(list1.index(list2[i]))

window


In [23]:
env.open_interactive_viewer()

In [17]:
env.model.geom_aabb[env.model.body('ycb-orange').geomadr][0]

array([-0.  , -0.  ,  0.  ,  0.04,  0.04,  0.04])

In [24]:
env.reset()

# object position lists on the table
obj_pos_list_table = {}
obj_pos_list_table['tray_gray'] = {'position':env.get_p_body('tray_gray'), 'size':np.array([0.17,0.07,0.28])}
obj_pos_list_table['kitchen-bowl-blue'] = {'position':env.get_p_body('kitchen-bowl-blue'), 'size': np.array([0.08,0.08,0.03])}
obj_pos_list_table['kitchen-bowl-red'] = {'position':env.get_p_body('kitchen-bowl-red'), 'size': np.array([0.08,0.08,0.03])}

obj_pos_list_table['ycb-apple-2'] = {'position':env.get_p_body('ycb-apple-2'), 'size': np.array([0.03,0.03,0.03])}
obj_pos_list_table['ycb-banana-2'] = {'position':env.get_p_body('ycb-banana-2'), 'size': np.array([0.02,0.04,0.01])}
obj_pos_list_table['ycb-lemon-2'] = {'position':env.get_p_body('ycb-lemon-2'), 'size': np.array([0.03,0.03,0.03])}
obj_pos_list_table['ycb-orange-2'] = {'position':env.get_p_body('ycb-orange-2'), 'size': np.array([0.04,0.04,0.04])}

print(obj_pos_list_table)
print(list(obj_pos_list_table.keys()))

{'tray_gray': {'position': array([0.9, 0. , 0.8]), 'size': array([0.17, 0.07, 0.28])}, 'kitchen-bowl-blue': {'position': array([ 0.9, -0.3,  0.8]), 'size': array([0.08, 0.08, 0.03])}, 'kitchen-bowl-red': {'position': array([0.9, 0.3, 0.8]), 'size': array([0.08, 0.08, 0.03])}, 'ycb-apple-2': {'position': array([0.9 , 0.3 , 0.82]), 'size': array([0.03, 0.03, 0.03])}, 'ycb-banana-2': {'position': array([0.9 , 0.03, 1.  ]), 'size': array([0.02, 0.04, 0.01])}, 'ycb-lemon-2': {'position': array([ 0.87, -0.07,  0.82]), 'size': array([0.03, 0.03, 0.03])}, 'ycb-orange-2': {'position': array([ 0.9 , -0.3 ,  0.82]), 'size': array([0.04, 0.04, 0.04])}}
['tray_gray', 'kitchen-bowl-blue', 'kitchen-bowl-red', 'ycb-apple-2', 'ycb-banana-2', 'ycb-lemon-2', 'ycb-orange-2']


### Get feasible_score_map

In [28]:
sys.path.append('../../../utils/')
from score_map import get_score_map, sample_pcd_from_score_map, plot_score_map

bandwidth = 0.01
nbins = 50
n_samples = 10

feasible_pcds = np.load('./data/without_shelf/feasible_pcd.npy')
score_map, di = get_score_map(feasible_pcds, nbins=nbins, bandwidth=bandwidth, PLOT=True)
sampled_physical = sample_pcd_from_score_map(score_map, feasible_pcds, di, nbins=nbins, num_samples=n_samples)
plot_score_map(score_map, feasible_pcds, nbins=nbins)

# Plotting
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
x, y, z = feasible_pcds[:, 0], feasible_pcds[:, 1], feasible_pcds[:, 2]

# Hide grid lines
# ax.grid(False)

ax.view_init(azim=0, elev=90)
# ax.set_xticks([])
# ax.set_yticks([])
# ax.set_zticks([])
plt.xlabel('X')
plt.ylabel('Y')
ax.set_zlabel('Z')
ax.scatter(x, y, z, c='r', marker='o')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './data/without_shelf/feasible_pcd.npy'

### Interact with GPT

In [ ]:
import os
import openai
import sys
sys.path.append('../../../')
from utils.gpt_helper import set_openai_api_key_from_txt,GPTchatClass,printmd
from utils.wiki_helper import wiki_search
from utils.util import printmd,extract_quoted_words
print ("openai version:[%s]"%(openai.__version__))

openai version:[0.27.8]


In [ ]:
# set_openai_api_key_from_txt(key_path='../../../key/rilab_key.txt')
set_openai_api_key_from_txt(key_path='../../../key/my_key.txt')
GPT = GPTchatClass(
    gpt_model='gpt-4', # 'gpt-3.5-turbo' / 'gpt-4'
    role_msg='Your are a helpful assistant summarizing infromation and answering user queries.')

OpenAI API Key Ready from [../../../key/my_key.txt].
Chat agent using [gpt-4] initialized with the follow role:[Your are a helpful assistant summarizing infromation and answering user queries.]


In [ ]:
object_description_list = ["plate", "book", "apple", "pockey"]
target_object_name = "plate"

user_msg = \
    f"""
    I will give you some scene descriptions, task descriptions and the user's preferred trajectory information:

    Scene description:
    There are objects of {list(obj_pos_list_table.keys())} on the table. Their respective positions and sizes are shown below.
    - {obj_pos_list_table}
    The table is located at [0.98,0,0.79], and the areas where objects can be placed on the table are shown below.
    - x: [0.65, 1.2]
    - y: [-0.38, 0.38]
    - z: [0.8, 0.9]

    Task description: 
    The task is to place the {target_object_name} on the front table.

    [Rules]
	1. The environment contains {list(obj_pos_list_table.keys())}. Do not invent new objects not listed here.
	2. The terminate condition is either when the manipulator has successfully placed the intended object into the bookshelf, ensuring it is stably settled, or if a critical failure occurs, such as knocking over a wine glass, dropping an object during transfer, or damaging an object or the bookshelf.
	3. You can assume that the robot can do anything, even for the most challenging task.
	4. Your plan should be as close to the provided template as possible. You can add additional information if you think it is necessary.
	5. Once you've gotten as much information as you think you need to perform the task, you can do it without asking any more questions, and you NEED to say, 'I get it.'. You should describe your plan in detail.
    6. You can say the region(area) where the object can be placed on the table. You should follow this format: [Object_Name], Spatial_Relationship_to_the_Object, use ONLY the list of objects I provided.
    7. The clusters of the regions are just for reference. But the categories of the regions are not fixed. You can change the categories of the regions if you think it is necessary.
	
    This is an example of instruction. For example, answer the output following instructions.
    For example, the answer can be like this:

    User: Based on the information I have provided, please recommend a suitable area to place {target_object_name} to accomplish the task successfully, and also let me know which region(area) is unsuitable to place it for the same purpose. Only you can say the region.
    GPT-4: Certainly! I have analyzed the provided positions and clustered them into five groups based on their spatial relationships and proximity to other objects in the table. 
    Here are the clusters:
    I recommend the region [red_bowl] to place the {target_object_name}. This area provides a balance between accessibility and stability, without interfering with other objects on the table. It's also aesthetically pleasing as it maintains symmetry on the table.
    And I recommend not to place the {target_object_name} in the region [blue_bowl]. This area is too close to the mug cup, which may cause the {target_object_name} to fall down when the robot is trying to place it.
    Summary: Recommended region: [red_bowl], Not recommended region: [blue_bowl]

    If you understand, Say "I understand" and I will start the simulation.
    """

In [ ]:
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=True,RETURN_RESPONSE=True)

[USER_MSG]



    I will give you some scene descriptions, task descriptions and the user's preferred trajectory information:

    Scene description:
    There are objects of ['kitchen-drainer', 'tray', 'mug_cup', 'kitchen-plate-2', 'kitchen-plate-3'] on the table. Their respective positions and sizes are shown below.
    - {'kitchen-drainer': {'position': array([0.9 , 0.35, 0.82]), 'size': array([0.17, 0.07, 0.28])}, 'tray': {'position': array([ 0.9, -0.3,  0.8]), 'size': array([0.11, 0.15, 0.01])}, 'mug_cup': {'position': array([ 0.9 , -0.3 ,  0.85]), 'size': array([0.05, 0.07, 0.1 ])}, 'kitchen-plate-2': {'position': array([0.9 , 0.29, 1.07]), 'size': array([0.1 , 0.1 , 0.01])}, 'kitchen-plate-3': {'position': array([0.9 , 0.23, 1.07]), 'size': array([0.1 , 0.1 , 0.01])}}
    The table is located at [0.98,0,0.79], and the areas where objects can be placed on the table are shown below.
    - x: [0.65, 1.2]
    - y: [-0.38, 0.38]
    - z: [0.8, 0.9]

    Task description: 
    The task is to place the plate on the front table.

    [Rules]
	1. The environment contains ['kitchen-drainer', 'tray', 'mug_cup', 'kitchen-plate-2', 'kitchen-plate-3']. Do not invent new objects not listed here.
	2. The terminate condition is either when the manipulator has successfully placed the intended object into the bookshelf, ensuring it is stably settled, or if a critical failure occurs, such as knocking over a wine glass, dropping an object during transfer, or damaging an object or the bookshelf.
	3. You can assume that the robot can do anything, even for the most challenging task.
	4. Your plan should be as close to the provided template as possible. You can add additional information if you think it is necessary.
	5. Once you've gotten as much information as you think you need to perform the task, you can do it without asking any more questions, and you NEED to say, 'I get it.'. You should describe your plan in detail.
    6. You can say the region(area) where the object can be placed on the table. You should follow this format: [Object_Name], Spatial_Relationship_to_the_Object, use ONLY the list of objects I provided.
    7. The clusters of the regions are just for reference. But the categories of the regions are not fixed. You can change the categories of the regions if you think it is necessary.
	
    This is an example of instruction. For example, answer the output following instructions.
    For example, the answer can be like this:

    User: Based on the information I have provided, please recommend a suitable area to place plate to accomplish the task successfully, and also let me know which region(area) is unsuitable to place it for the same purpose. Only you can say the region.
    GPT-4: Certainly! I have analyzed the provided positions and clustered them into five groups based on their spatial relationships and proximity to other objects in the table. 
    Here are the clusters:
    I recommend the region [Dish_Rack] to place the plate. This area provides a balance between accessibility and stability, without interfering with other objects on the table. It's also aesthetically pleasing as it maintains symmetry on the table.
    And I recommend not to place the plate in the region [Mug_Cup]. This area is too close to the mug cup, which may cause the plate to fall down when the robot is trying to place it.
    Summary: Recommended region: [Dish_Rack], Not recommended region: [Mug_Cup]

    If you understand, Say "I understand" and I will start the simulation.
    

[GPT_OUTPUT]


Based on the information given, here are my area recommendations and decisions for the placing of the plate:

Recommended Area:
After analyzing the placements of all objects and their positions and sizes, I would recommend placing the plate in the 'kitchen-drainer' region as it has enough space and it's in a stable location. It also has an easy reach for the robot arm and wouldn't interfere with the other objects.

Unsuitable Area:
The 'mug_cup' region is unsuitable due to its close proximity to the mug cup which poses an increased risk of the plate falling over or colliding with the mug during placement. 

Summary: Recommended region: [kitchen-drainer], Not recommended region: [mug_cup]. 

Once you confirm the understanding of these instructions, I can begin with the simulation.

In [ ]:
import re

text = response_content
pattern_recommended = r'recommended region: \[(.*?)\]'
pattern_not_recommended = r'not recommended region: \[(.*?)\]'

# Use re.search to find the first occurrence of the patterns
recommended_region = re.search(pattern_recommended, text.lower())
not_recommended_region = re.search(pattern_not_recommended, text.lower())

# Extract the matched groups
recommended_region = recommended_region.group(1) if recommended_region else None
not_recommended_region = not_recommended_region.group(1) if not_recommended_region else None

recommended_region, not_recommended_region

('kitchen-drainer', 'mug_cup')